In [ ]:
import os
import random
import pandas as pd
import numpy as np
import re
import json

import matplotlib.pyplot as plt
import seaborn as sns
from script.timestamp_smoothing import process_timestamp_correction,detect_jitter_patterns

from script.data_structures import BaseInfo, ACC_Raw_Data, WearInfo, PPG_Raw_Data, TYHX_PPG_Data
from script.data_types import get_full_data_type
from script.common import (
    get_raw_data, 
    parse_filename, 
    expand_acc_data, 
    expand_ppg_data, 
    merge_sensor_data,
    expand_sensor_data,
    process_and_merge_sensor_data
)
from script.wear_detection_watch_dll import *

# 设置matplotlib参数
plt.style.use('default')  # 使用默认样式

# 尝试设置中文字体支持(如果失败则使用默认字体)
try:
    plt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei', 'DejaVu Sans']
    plt.rcParams['axes.unicode_minus'] = False
except:
    print("注意: 中文字体设置失败,将使用默认字体")
    pass

In [3]:
def run_wear_detection_algorithm(df, directory,file_name_without_ext):
    # 1. 创建一个列表用于存储日志
    log_buffer = []

    # 2. 定义回调：只存列表,不写文件
    def memory_logging_callback(message):
        # print(message, end='') 
        log_buffer.append(message) # 存入内存
    
    dll_path = r"G:\AlgorithmLibrary\01_alg_wearing\build\bin\libalg_wear.dll"
    results = []
    try:
        # 使用上下文管理器自动管理DLL资源
        with WearDetectionWatchDLL(dll_path, print_callback=memory_logging_callback) as dll:
            
            # 获取版本
            version = dll.get_version()
            print(f"alg_wear_new Ver: {version.major}.{version.minor}.{version.patch}")

            config = AlgWearInitParament()
            # 根据 alg_wear_watch.h 注释,建议 8316 3690 3918 用 5000
            config.ppg_low_ir_threshold = 30000 
            config.wear_amb_std_threshold = 0
            config.acc_quite_time = 0 # 
            config.ppg_unwear_cnt = 1 # 默认5
            config.green_confusion_threshold = 0
            config.confirmed_recheck_timer = 300
            config.is_ring_flag = 0 # 0否,1是
            dll.init_with_config(config)


            for idx, row in df.iterrows():
                
                
                # 1. 准备 ACC
                acc_data = row['acc_data']
                acc_axis = dll.prepare_acc_data(acc_data)
                unix_timestamp = row['unix_timestamp']
                # 2. 准备 PPG
                ppg_g = row['ppg_g_data']
                ppg_ir = row['ppg_ir_data']
                ppg_r = row['ppg_r_data']
                # current = row['data'][1]
                # print(f"Processing row {idx}", )
                # print(f"  ACC data: {acc_data}")
                # print(f"  PPG G data: {ppg_g}")
                # print(f"  PPG IR data: {ppg_ir}")

                # 构造 ppg_data 字典
                ppg_config = {
                    'g': ppg_g,
                    'r' :ppg_r,
                    'low_ir': ppg_ir, # 使用 IR 数据
                    'channel_num': 1,
                    # 'g_current': current, # 模拟电流值
                    'fs': 32, 
                }
                ppg_sensors = dll.prepare_ppg_data(ppg_config)

                # 3. 准备 WearingData
                wearing_data = WearingData()
                wearing_data.measure_type = WearMeasureType.WEAR_IR
                wearing_data.spo2_alg_ir_result = 1
                wearing_data.G_LED = 1
                wearing_data.ppg_g_reg_led_value = 0
                wearing_data.freq_snr = 255

                # 4. 执行
                result = dll.process(acc_axis=acc_axis, ppg_sensors=ppg_sensors, wearing_data=wearing_data)
                result_dict = dll.result_to_dict(result)
                result_dict['unix_timestamp'] = unix_timestamp
                # 打印简单的结果验证
                # state_str = "WEAR" if result_dict.get('is_wear') else "UNWEAR" # 
                # print(f"-> Done. Result: {result_dict}")
                
                results.append(result_dict)

            print(f"\nSuccessfully processed {len(results)} rows.")

    
    except Exception as e:
        print(f"Error running wear detection: {e}")

    finally:
        # 4. 【核心】在 finally 块中将内存中的日志一次性写入文件
        # 这样无论是否报错,日志都会被保存
        log_file_path = os.path.join(directory, f"{file_name_without_ext}_log.txt")
        try:
            with open(log_file_path, 'w', encoding='utf-8') as f:
                # 将列表中的字符串拼接后写入,或者逐行写入
                # 注意：如果 message 自带换行符,就不用加 \n
                for msg in log_buffer:
                    f.write(msg) 
            print(f"日志已保存至: {log_file_path}")
        except Exception as io_e:
            print(f"保存日志失败: {io_e}")

            
    return results



In [4]:
def plot_wear_detection_results(expanded_ppg_ir_df, expanded_ppg_g_df, expanded_ppg_r_df, expanded_acc_df, wear_df, directory, file_name_without_ext, is_show=False, result_type='original'):
    """
    绘制佩戴检测结果数据,包括展开的PPG IR、PPG Green、PPG Red、加速度数据和佩戴标志。
    根据可用数据动态调整子图数量和布局。
    """
    # 检查每个数据源是否有效
    has_ppg_ir = expanded_ppg_ir_df is not None and not expanded_ppg_ir_df.empty
    has_ppg_g = expanded_ppg_g_df is not None and not expanded_ppg_g_df.empty
    has_ppg_r = expanded_ppg_r_df is not None and not expanded_ppg_r_df.empty
    has_acc = expanded_acc_df is not None and not expanded_acc_df.empty
    has_wear = wear_df is not None and not wear_df.empty

    # 计算需要的子图数量(PPG图作为一个子图)
    plot_count = sum([has_ppg_ir or has_ppg_g or has_ppg_r, has_acc, has_wear])

    if plot_count == 0:
        print("没有可用的数据进行绘制")
        return

    fig, axes = plt.subplots(plot_count, 1, figsize=(20, 4 * plot_count), sharex=True)
    if plot_count == 1:
        axes = [axes]

    result_label = "Original" if result_type == 'original' else "Simulated"
    fig.suptitle(f"{file_name_without_ext} - {result_label}", fontsize=16, fontweight='bold')
    current_ax_idx = 0

    # 绘制 PPG 数据(IR / Green / Red)
    if has_ppg_ir or has_ppg_g or has_ppg_r:
        ax = axes[current_ax_idx]

        # 为了清晰,使用双 Y 轴：左轴绘制 IR 和 Red,右轴绘制 Green(如果存在)
        ax_twinx = None
        if has_ppg_ir:
            ax.plot(expanded_ppg_ir_df['unix_timestamp'], expanded_ppg_ir_df['ppg_value'], label='PPG IR', color='pink', linewidth=1)
        if has_ppg_r:
            ax.plot(expanded_ppg_r_df['unix_timestamp'], expanded_ppg_r_df['ppg_value'], '.',markersize=1, label='PPG Red', color='red', linewidth=1)
        if has_ppg_g:
            ax_twinx = ax.twinx()
            ax_twinx.set_ylabel('PPG Green', fontsize=12)
            # ax_twinx.plot(expanded_ppg_g_df['unix_timestamp'], expanded_ppg_g_df['ppg_value'], '.',markersize=1, label='PPG Green', color='green', linewidth=0.5) # type: ignore
            ax_twinx.plot(expanded_ppg_g_df['unix_timestamp'], expanded_ppg_g_df['ppg_value'], label='PPG Green', color='green', linewidth=0.5) # type: ignore

        ax.set_title('PPG Data')
        ax.set_ylabel('PPG (IR/Red)')
        ax.grid(True, alpha=0.3)

        # 合并图例
        handles, labels = ax.get_legend_handles_labels()
        if ax_twinx is not None:
            h2, l2 = ax_twinx.get_legend_handles_labels()
            handles += h2
            labels += l2
        if handles:
            ax.legend(handles, labels, loc='upper left')

        current_ax_idx += 1

    # 绘制加速度数据
    if has_acc:
        ax = axes[current_ax_idx]
        ax.plot(expanded_acc_df['unix_timestamp'], expanded_acc_df['x'], label='ACC X', color='red', linewidth=0.5)
        ax.plot(expanded_acc_df['unix_timestamp'], expanded_acc_df['y'], label='ACC Y', color='green', linewidth=0.5)
        ax.plot(expanded_acc_df['unix_timestamp'], expanded_acc_df['z'], label='ACC Z', color='blue', linewidth=0.5)
        ax.set_ylabel('Acceleration')
        ax.set_title('Acceleration Data')
        ax.legend()
        ax.grid(True, alpha=0.3)
        current_ax_idx += 1

    # 绘制佩戴标志
    if has_wear:
        ax = axes[current_ax_idx]
        ax.plot(wear_df['unix_timestamp'], wear_df['wear_flag'], marker='o', markersize=3, linewidth=1, label='Wear Flag', color='orange')
        ax.set_ylabel('Wear Flag')
        ax.set_xlabel('Unix Timestamp (s)')
        ax.set_title('Wear Info Flags')
        ax.legend()
        ax.grid(True, alpha=0.3)

    suffix = "_original" if result_type == 'original' else "_simulated"
    output_file = f"{directory}/{file_name_without_ext}_wear{suffix}.png"

    plt.savefig(output_file, dpi=300)
    print(f"图形已保存至: {output_file}")
    plt.tight_layout()
    if is_show:
        plt.show()
    else:
        plt.close()


In [5]:
%matplotlib qt

is_ring = False

# G:\PyScript\rawdata\Wear\CWR12G_0122  客户测试
# HeartRate_Static_TEST_20260122_110331_de54.rawdata X
# HeartRate_Static_TEST_20260122_112656_de54.rawdata        电流 92   offset 15
# HeartRate_Static_TEST_20260122_111122_de54.rawdata 女 偏黑   电流164   offset 15
# HeartRate_Static_TEST_20260122_111658_de54.rawdata 男 黑    电流 254   offset 5
# HeartRate_Static_TEST_20260122_111902_de54.rawdata         电流 254   offset 10
# HeartRate_Static_TEST_20260122_112111_de54.rawdata         电流 254   offset 10
# HeartRate_Static_TEST_20260122_113016_de54.rawdata        电流 254   offset 5
# HeartRate_Static_TEST_20260122_113219_de54.rawdata        电流 254   offset 5
# HeartRate_Static_TEST_20260122_113439_de54.rawdata        电流 254   offset 5

# directory = r'G:\PyScript\rawdata\Wear\CWR12G_0122'
# file_name = "HeartRate_Static_TEST_20260122_113219_de54.rawdata"


# G:\PyScript\rawdata\Wear\BUG\One_key
# HeartRate_Static_Charan_20260121_123414_b27b.rawdata  # 
# HeartRate_Static_MuraliM_20260121_123019_5e7a.rawdata #  一键测量失败 佩戴失效
# HeartRate_Static_Maruthi_20260121_123210_f163.rawdata # 呼吸率 HRV

# directory = r'G:\PyScript\rawdata\Wear\BUG\One_key'
# file_name = "HeartRate_Static_MuraliM_20260121_123019_5e7a.rawdata"

# Sleep_Normal_mz_20260123_103908_b983  # 手表放床上出佩戴 非常好样本 1
# Sleep_Normal_mz_20260123_103908_bacb  # 手表放床上出佩戴 非常好样本 2
directory = r'G:\PyScript\rawdata\Wear\BUG'
file_name = "Sleep_Normal_mz_20260123_103908_b983.rawdata"

file_path = os.path.join(directory, file_name)

real_path = os.path.realpath(file_path)
file_name = os.path.basename(file_path)
file_name_without_ext, ext = os.path.splitext(file_name)
directory = os.path.dirname(real_path)
info =  parse_filename(file_name)
mac = info.get('mac','0000').lower()

print(f"real_path: {real_path} {directory} {file_name_without_ext}{ext}  {mac} \n")

result = get_raw_data(real_path,is_ring=is_ring)

save_dir = os.path.join(directory, "data")
os.makedirs(save_dir, exist_ok=True)


data_type_dict = result.get('data_types', {})

base_info_df = result.get('base_info_df', pd.DataFrame(columns=['unix_timestamp']))
wear_info_df = result.get('wear_info_df', pd.DataFrame(columns=['unix_timestamp']))
tyhx_df = result.get('TYHX_data_df', pd.DataFrame(columns=['unix_timestamp']))
tyhx_agc_data_df = result.get('TYHX_agc_data_df', pd.DataFrame(columns=['unix_timestamp']))

ppg_g_df = result.get('ppg_g_df',pd.DataFrame(columns=['unix_timestamp','data','ppg_data']))
ppg_r_df = result.get('ppg_r_df',pd.DataFrame(columns=['unix_timestamp','data','ppg_data']))
ppg_ir_df = result.get('ppg_ir_df',pd.DataFrame(columns=['unix_timestamp','data','ppg_data']))
ppg_r_ir_df = result.get('ppg_r_ir_df',pd.DataFrame(columns=['unix_timestamp','data','ppg_data']))
acc_df =  result.get('acc_df', pd.DataFrame(columns=['unix_timestamp']))


sorted_data_types = sorted(data_type_dict.keys(), key=lambda x: int(x, 16))
readable_types = [f"{dt}  {get_full_data_type(dt)}  count: {data_type_dict[dt]} " for dt in sorted_data_types] 
print(f"文件 {file_path} 解析完成,包含数据类型: ")
print(f"{"\n".join(readable_types)}")

(combined_ppg_df, 
wear_info_df,
expanded_acc_df, 
expanded_ppg_g_df, 
expanded_ppg_ir_df, 
expanded_ppg_r_df)= process_and_merge_sensor_data(acc_df, ppg_g_df, ppg_ir_df,ppg_r_df, wear_info_df) 


expanded_ppg_r_ir_df = expand_ppg_data(ppg_r_ir_df, 'ppg_data')

# tyhx_df.to_csv(f"{save_dir}/{file_name_without_ext}_tyhx_data.csv", index=False, encoding='utf-8')
# tyhx_agc_data_df.to_csv(f"{save_dir}/{file_name_without_ext}_tyhx_agc_data.csv", index=False, encoding='utf-8')
# expanded_ppg_r_ir_df.to_csv(f"{save_dir}/{file_name_without_ext}_expanded_ppg_r_ir.csv", index=False, encoding='utf-8')
# combined_ppg_df.to_csv(f"{save_dir}/{file_name_without_ext}_combined_ppg.csv", index=False, encoding='utf-8')
# expanded_ppg_g_df.to_csv(f"{save_dir}/{file_name_without_ext}_expanded_ppg_g.csv", index=False, encoding='utf-8')
# expanded_ppg_r_df.to_csv(f"{save_dir}/{file_name_without_ext}_expanded_ppg_r.csv", index=False, encoding='utf-8')
# expanded_ppg_ir_df.to_csv(f"{save_dir}/{file_name_without_ext}_expanded_ppg_ir.csv", index=False, encoding='utf-8')


null_df = pd.DataFrame()

# # 调用函数绘制结果
plot_wear_detection_results(expanded_ppg_ir_df, 
                            expanded_ppg_g_df, 
                            expanded_ppg_r_df, 
                            expanded_acc_df, 
                            wear_info_df, 
                            save_dir, 
                            file_name_without_ext,
                            is_show=True,result_type='original')

real_path: G:\PyScript\rawdata\Wear\BUG\Sleep_Normal_mz_20260123_103908_b983.rawdata G:\PyScript\rawdata\Wear\BUG Sleep_Normal_mz_20260123_103908_b983.rawdata  b983 

文件 G:\PyScript\rawdata\Wear\BUG\Sleep_Normal_mz_20260123_103908_b983.rawdata 解析完成,包含数据类型: 
00  BASE_INFO             基础信息                   count: 603 
15  RAW_PPG_GREEN         原始PPG绿光                count: 601 
16  RAW_PPG_RED           原始PPG红光                count: 24 
17  RAW_PPG_IR            原始PPG红外                count: 601 
18  RAW_ACC               原始加速度                  count: 1206 
1d  RAW_EDA               原始皮肤电导                 count: 38 
21  RAW_ACC_TO_PPG        原始加速度到PPG              count: 1203 
26  RAW_PPG_RED_IR        原始PPG红灯红外              count: 24 
33  AMBIENT_INFO          环境信息                   count: 577 
67  SLEEP_INFO            睡眠信息                   count: 603 
68  HRV_INFO              心率变异性信息                count: 600 
69  MOTION_INFO           运动识别                   count: 603 
71  WEAR_IN

In [11]:
def create_wear_detection_df(ppg_g_df, ppg_r_df, ppg_ir_df, acc_df, start, end):
    """
    组合 PPG 和 ACC 数据,形成适合 run_wear_detection_algorithm 的 DataFrame。
    
    参数:
    - ppg_g_df: PPG Green 数据 DataFrame,包含 'unix_timestamp' 和 'ppg_data'
    - ppg_r_df: PPG Red 数据 DataFrame,包含 'unix_timestamp' 和 'ppg_data'
    - ppg_ir_df: PPG IR 数据 DataFrame,包含 'unix_timestamp' 和 'ppg_data'
    - acc_df: ACC 数据 DataFrame,包含 'unix_timestamp' 和 'acc_data'
    - start: 开始时间戳 (unix_timestamp)
    - end: 结束时间戳 (unix_timestamp)
    
    返回:
    - 合并后的 DataFrame,包含列: unix_timestamp, acc_data, ppg_g_data, ppg_ir_data, ppg_r_data
    """
    # 重命名列以匹配 run_wear_detection_algorithm 的期望
    ppg_g_renamed = ppg_g_df[['unix_timestamp','ppg_data_0']].copy() \
                   .rename(columns={'ppg_data_0': 'ppg_g_data'}) \
                   .drop_duplicates(subset=['unix_timestamp'])
    ppg_r_renamed = ppg_r_df[['unix_timestamp','ppg_data_0']].copy() \
                   .rename(columns={'ppg_data_0': 'ppg_r_data'}) \
                   .drop_duplicates(subset=['unix_timestamp'])
    ppg_ir_renamed = ppg_ir_df[['unix_timestamp','ppg_data_0']].copy() \
                   .rename(columns={'ppg_data_0': 'ppg_ir_data'}) \
                   .drop_duplicates(subset=['unix_timestamp'])
    acc_df_copy = acc_df[['unix_timestamp','acc_data']].copy() \
                   .drop_duplicates(subset=['unix_timestamp'])

    

    # 过滤时间范围
    ppg_g_filtered = ppg_g_renamed[(ppg_g_renamed['unix_timestamp'] >= start) & (ppg_g_renamed['unix_timestamp'] <= end)]
    ppg_r_filtered = ppg_r_renamed[(ppg_r_renamed['unix_timestamp'] >= start) & (ppg_r_renamed['unix_timestamp'] <= end)]
    ppg_ir_filtered = ppg_ir_renamed[(ppg_ir_renamed['unix_timestamp'] >= start) & (ppg_ir_renamed['unix_timestamp'] <= end)]
    acc_filtered = acc_df_copy[(acc_df_copy['unix_timestamp'] >= start) & (acc_df_copy['unix_timestamp'] <= end)]
    
    # 合并 DataFrame,使用 outer join 以包含所有时间戳
    merged_df = ppg_g_filtered.merge(ppg_r_filtered, on='unix_timestamp', how='outer')
    merged_df = merged_df.merge(ppg_ir_filtered, on='unix_timestamp', how='outer')
    merged_df = merged_df.merge(acc_filtered, on='unix_timestamp', how='outer')
    
    # 排序并重置索引
    merged_df = merged_df.sort_values('unix_timestamp').reset_index(drop=True)
    
    # 确保所有必需列存在,如果缺失则填充 NaN
    required_columns = ['unix_timestamp', 'acc_data', 'ppg_g_data', 'ppg_ir_data', 'ppg_r_data']
    for col in required_columns:
        if col not in merged_df.columns:
            merged_df[col] = np.nan
    
    return merged_df[required_columns]

In [12]:
# 1768998625
# 1768998652

# ppg_g_df  ppg_data_0 -> ppg_g_data
# ppg_r_df  ppg_data_0 ->ppg_ir_data
# ppg_ir_df ppg_data_0 ->ppg_ir_data
# acc_df    acc_data

# 示例调用
wear_df = create_wear_detection_df(ppg_g_df, ppg_r_df, ppg_ir_df, acc_df, 1768998625, 1768998652)
results = run_wear_detection_algorithm(wear_df, directory, file_name_without_ext)

alg_wear_new Ver: 4.21.9
Error running wear detection: object of type 'float' has no len()
日志已保存至: G:\PyScript\rawdata\Wear\BUG\One_key\HeartRate_Static_MuraliM_20260121_123019_5e7a_log.txt


In [17]:
ppg_g_renamed = ppg_g_df[['unix_timestamp','ppg_data_0']].copy() \
                .rename(columns={'ppg_data_0': 'ppg_g_data'}) 
                # \.drop_duplicates(subset=['unix_timestamp'])
ppg_r_renamed = ppg_r_df[['unix_timestamp','ppg_data_0']].copy() \
                .rename(columns={'ppg_data_0': 'ppg_r_data'}) 
                # .drop_duplicates(subset=['unix_timestamp'])
ppg_ir_renamed = ppg_ir_df[['unix_timestamp','ppg_data_0']].copy() \
                .rename(columns={'ppg_data_0': 'ppg_ir_data'}) 
                # .drop_duplicates(subset=['unix_timestamp'])
acc_df_copy = acc_df[['unix_timestamp','acc_data']].copy() 
                # .drop_duplicates(subset=['unix_timestamp'])

In [18]:
acc_df_copy['unix_timestamp'].to_clipboard()

In [ ]:
df = process_timestamp_correction(acc_df)

In [8]:

# acc_df: 加速度数据 DataFrame
# ppg_g_df: 绿光 PPG 数据 DataFrame
# ppg_ir_df: 红外光 PPG 数据 DataFrame
# base_info_df: 基础信息 DataFrame
# wear_info_df: 佩戴信息 DataFrame

--- 1. 多包堆积检测 (Bursts) ---
 unix_timestamp  count row_indices
     1769164777      2    [29, 30]

--- 2. 时间空缺检测 (Gaps) ---
row_pair  gap_start_unix  missing_seconds
(30, 31)      1769164778                1


时间戳平滑处理流程 (tolerance=4, 分段阈值=8)

[步骤 1/3] 数据分段中...

[步骤 2/3] 生成平滑骨架...

[步骤 3/3] 应用修正到原始行...

✅ 处理完成



In [ ]:
df